# A1111 Model Installer — SDXL (UI)

Install **Stable Diffusion XL** models and common extension assets into the WebUI data directory.

**Folders (under `WEBUI_ROOT`, default `/workspace/a1111`):**
- **Checkpoints (SDXL Base/Refiner)** → `models/Stable-diffusion/`
- **VAE (SDXL)** → `models/VAE/`
- **ControlNet (SDXL)** → `models/ControlNet/`
- **ADetailer YOLO** → `models/adetailer/`
- **ESRGAN** → `models/ESRGAN/`
- **IP-Adapter (SDXL)** → `models/ControlNet/` (used via ControlNet)
- **AnimateDiff (SDXL)** → `models/AnimateDiff/`
- **ReActor (InsightFace)** → `models/insightface/`
- **Embeddings** → `embeddings/`
- **LoRA (SDXL)** → `models/Lora/`

Re-run any time — existing files are skipped.

> Optional: set `HF_TOKEN` (for gated HF files) and/or `CIVITAI_TOKEN` (for private/authorized Civitai links). Public files work without tokens.

In [ ]:
from pathlib import Pathimport osROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))CKPT_DIR    = ROOT / "models/Stable-diffusion"VAE_DIR     = ROOT / "models/VAE"CN_DIR      = ROOT / "models/ControlNet"AD_DIR      = ROOT / "models/adetailer"ESRGAN_DIR  = ROOT / "models/ESRGAN"IPAD_DIR    = CN_DIRANIM_DIR    = ROOT / "models/AnimateDiff"REACT_DIR   = ROOT / "models/insightface"EMB_DIR     = ROOT / "embeddings"LORA_DIR    = ROOT / "models/Lora"for d in (CKPT_DIR, VAE_DIR, CN_DIR, AD_DIR, ESRGAN_DIR, IPAD_DIR, ANIM_DIR, REACT_DIR, EMB_DIR, LORA_DIR):    d.mkdir(parents=True, exist_ok=True)print("Data root:", ROOT)print("↳ Checkpoints:", CKPT_DIR)print("↳ VAE:", VAE_DIR)print("↳ ControlNet + IP-Adapter:", CN_DIR)print("↳ ADetailer:", AD_DIR)print("↳ ESRGAN:", ESRGAN_DIR)print("↳ AnimateDiff:", ANIM_DIR)print("↳ ReActor (InsightFace):", REACT_DIR)print("↳ Embeddings:", EMB_DIR)print("↳ LoRA:", LORA_DIR)

## Helpers (progress + smart filenames)
Checkbox items can be **Hugging Face** (repo+filename) or a **direct URL** (e.g., Civitai). The downloader auto-detects and uses the right method.

In [ ]:
import os, re, shutil, requestsfrom urllib.parse import urlparsefrom tqdm import tqdmfrom huggingface_hub import hf_hub_downloadfrom pathlib import Pathdef human(n: int) -> str:    for u in ("B","KB","MB","GB"):        if n < 1024: return f"{n:.1f} {u}"        n /= 1024    return f"{n:.1f} TB"def _save_stream_to(dest: Path, r: requests.Response) -> Path:    dest.parent.mkdir(parents=True, exist_ok=True)    if dest.exists():        print("✔ Exists:", dest); return dest    total = int(r.headers.get("Content-Length", 0))    tmp = dest.with_suffix(dest.suffix + ".part")    with open(tmp, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=dest.name, ncols=80) as bar:        for chunk in r.iter_content(chunk_size=1024*1024):            if chunk:                f.write(chunk); bar.update(len(chunk))    tmp.replace(dest)    print(f"✔ Saved: {dest} ({human(dest.stat().st_size)})")    return destdef _filename_from_headers_or_url(r: requests.Response, url: str, default_ext: str = ".safetensors") -> str:    cd = r.headers.get("Content-Disposition", "")    m = re.search(r'filename="?([^";]+)"?', cd)    if m:        return m.group(1)    name = url.split("/")[-1].split("?", 1)[0]    if "." not in name:        name += default_ext    return namedef download_direct(url: str, dest_dir: Path, dest_name: str|None=None) -> Path:    # Optional Civitai bearer token (works without it if public)    headers = {"User-Agent": "sd-webui-hub-notebook/1.0"}    netloc = urlparse(url).netloc.lower()    civitai_token = os.environ.get("CIVITAI_TOKEN")    if "civitai.com" in netloc and civitai_token:        headers["Authorization"] = f"Bearer {civitai_token}"    r = requests.get(url, stream=True, allow_redirects=True, headers=headers)    r.raise_for_status()    filename = dest_name or _filename_from_headers_or_url(r, url)    return _save_stream_to(dest_dir/filename, r)def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None) -> Path:    dest = dest_dir / (dest_name or filename)    if dest.exists():        print("✔ Exists:", dest); return dest    print(f"→ HF: {repo_id}/{filename}")    path = hf_hub_download(        repo_id=repo_id,        filename=filename,        token=os.environ.get("HF_TOKEN"),  # optional, works without for public        resume_download=True    )    shutil.copy2(path, dest)    print(f"✔ HF saved: {dest} ({human(dest.stat().st_size)})")    return destdef fetch(item: dict, dest_dir: Path) -> None:    """Item can be {name, hf:{repo,filename}} OR {name, url, dest_name?}."""    if "hf" in item:        repo, fname = item["hf"]["repo"], item["hf"]["filename"]        download_hf(repo, fname, dest_dir, item.get("dest_name"))    elif "url" in item:        download_direct(item["url"], dest_dir, item.get("dest_name"))    else:        raise ValueError("Item must contain 'hf' or 'url'.")def download_extras(lines: str, dest_dir: Path) -> None:    for url in [u.strip() for u in lines.splitlines() if u.strip()]:        try:            download_direct(url, dest_dir)        except Exception as e:            print("✖", url, "→", e)

## Pick what to install
Each section has checkboxes **and** an **Extra URLs** box (one URL per line).

In [ ]:
import ipywidgets as Wfrom IPython.display import display, Markdown# ---------- SDXL CHECKPOINTS ----------ckpt_items = [  {"name": "SDXL Base 1.0 (official)",   "hf": {"repo": "stabilityai/stable-diffusion-xl-base-1.0", "filename": "sd_xl_base_1.0.safetensors"}},  {"name": "SDXL Refiner 1.0 (official)",   "hf": {"repo": "stabilityai/stable-diffusion-xl-refiner-1.0", "filename": "sd_xl_refiner_1.0.safetensors"}}]ckpt_pre = {"SDXL Base 1.0 (official)"}ckpt_checks = [W.Checkbox(description=i["name"], value=(i["name"] in ckpt_pre)) for i in ckpt_items]ckpt_extra = W.Textarea(placeholder="Extra SDXL checkpoint URLs (.safetensors/.ckpt) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- VAE (SDXL) ----------vae_items = [  {"name": "SDXL VAE (fp16 fix)", "hf": {"repo": "madebyollin/sdxl-vae-fp16-fix", "filename": "sdxl_vae.safetensors"}},  {"name": "SDXL VAE (stabilityai)", "hf": {"repo": "stabilityai/sdxl-vae", "filename": "sdxl-vae-fp16-fix.safetensors"}}]vae_pre = {"SDXL VAE (fp16 fix)"}vae_checks = [W.Checkbox(description=i["name"], value=(i["name"] in vae_pre)) for i in vae_items]vae_extra = W.Textarea(placeholder="Extra SDXL VAE URLs (.safetensors/.ckpt) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- ControlNet (SDXL) ----------cn_items = [  {"name": "Canny (SDXL)",    "hf": {"repo": "diffusers/controlnet-canny-sdxl-1.0",    "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "controlnet-canny-sdxl-1.0.safetensors"},  {"name": "Depth (SDXL)",    "hf": {"repo": "diffusers/controlnet-depth-sdxl-1.0",    "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "controlnet-depth-sdxl-1.0.safetensors"},  {"name": "OpenPose (SDXL)", "hf": {"repo": "diffusers/controlnet-openpose-sdxl-1.0", "filename": "diffusion_pytorch_model.safetensors"}, "dest_name": "controlnet-openpose-sdxl-1.0.safetensors"}]cn_pre = {"Canny (SDXL)"}cn_checks = [W.Checkbox(description=i["name"], value=(i["name"] in cn_pre)) for i in cn_items]cn_extra = W.Textarea(placeholder="Extra ControlNet SDXL URLs (.safetensors/.ckpt) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- ADetailer YOLO ----------ad_items = [  {"name": "face_yolov8n.pt", "url": "https://huggingface.co/Bing-su/adetailer/resolve/main/face_yolov8n.pt"},  {"name": "hand_yolov8n.pt", "url": "https://huggingface.co/Bing-su/adetailer/resolve/main/hand_yolov8n.pt"},  {"name": "face_yolov8s.pt", "url": "https://huggingface.co/Bing-su/adetailer/resolve/main/face_yolov8s.pt"},  {"name": "hand_yolov8s.pt", "url": "https://huggingface.co/Bing-su/adetailer/resolve/main/hand_yolov8s.pt"}]ad_pre = {"face_yolov8n.pt","hand_yolov8n.pt"}ad_checks = [W.Checkbox(description=i["name"], value=(i["name"] in ad_pre)) for i in ad_items]ad_extra = W.Textarea(placeholder="Extra ADetailer model URLs (.pt) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- ESRGAN ----------esr_items = [  {"name": "4x-UltraSharp.pth", "hf": {"repo": "Coyote-A/4x-UltraSharp", "filename": "4x-UltraSharp.pth"}}]esr_pre = {"4x-UltraSharp.pth"}esr_checks = [W.Checkbox(description=i["name"], value=(i["name"] in esr_pre)) for i in esr_items]esr_extra = W.Textarea(placeholder="Extra ESRGAN URLs (.pth) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- IP-Adapter (SDXL, stored under ControlNet) ----------ipad_items = [  {"name": "ip-adapter_sdxl.safetensors",       "hf": {"repo": "h94/IP-Adapter", "filename": "models/ip-adapter_sdxl.safetensors"},       "dest_name": "ip-adapter_sdxl.safetensors"},  {"name": "ip-adapter-plus_sdxl.safetensors", "hf": {"repo": "h94/IP-Adapter", "filename": "models/ip-adapter-plus_sdxl.safetensors"}, "dest_name": "ip-adapter-plus_sdxl.safetensors"}]ipad_pre = {"ip-adapter_sdxl.safetensors"}ipad_checks = [W.Checkbox(description=i["name"], value=(i["name"] in ipad_pre)) for i in ipad_items]ipad_extra = W.Textarea(placeholder="Extra IP-Adapter SDXL URLs (.safetensors/.bin) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- AnimateDiff (SDXL) ----------anim_items = [  {"name": "mm_sdxl_v10.ckpt (example)", "url": "https://huggingface.co/guoyww/animatediff/resolve/main/mm_sdxl_v10.ckpt"}]anim_pre = set()anim_checks = [W.Checkbox(description=i["name"], value=(i["name"] in anim_pre)) for i in anim_items]anim_extra = W.Textarea(placeholder="Extra AnimateDiff SDXL URLs (.ckpt/.safetensors) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- ReActor (InsightFace) ----------REACT_MODELS_DIR   = REACT_DIR / "models"REACT_ANTELOPE_DIR = REACT_DIR / "antelopev2"REACT_MODELS_DIR.mkdir(parents=True, exist_ok=True)REACT_ANTELOPE_DIR.mkdir(parents=True, exist_ok=True)react_items = []react_checks = []react_extra = W.Textarea(placeholder="Extra InsightFace/antelopev2 URLs — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- Embeddings (SDXL-capable) ----------emb_items = [  {"name": "EasyNegative (example)", "url": "https://civitai.com/api/download/models/9208?type=TextualInversion&format=SafeTensor", "dest_name": "EasyNegative.safetensors"}]emb_pre = set()emb_checks = [W.Checkbox(description=i["name"], value=(i["name"] in emb_pre)) for i in emb_items]emb_extra = W.Textarea(placeholder="Extra Embedding URLs (.pt/.bin/.safetensors) — one per line", layout=W.Layout(width="100%", height="80px"))# ---------- LoRA (SDXL) ----------lora_items = [  {"name": "Example SDXL LoRA #1", "url": "https://civitai.com/api/download/models/000000?type=Model&format=SafeTensor", "dest_name": "sdxl_lora_1.safetensors"}]lora_pre = set()lora_checks = [W.Checkbox(description=i["name"], value=(i["name"] in lora_pre)) for i in lora_items]lora_extra = W.Textarea(placeholder="Extra SDXL LoRA URLs (.safetensors/.ckpt) — one per line", layout=W.Layout(width="100%", height="80px"))btn = W.Button(description="Download Selected", button_style="success", icon="download")out = W.Output()def on_click(_):    out.clear_output()    with out:        errors = []        # SDXL Checkpoints        for cb, item in zip(ckpt_checks, ckpt_items):            if not cb.value: continue            try:                item = dict(item)                if "dest_name" not in item and "hf" in item:                    item["dest_name"] = item["hf"]["filename"].split("/")[-1]                fetch(item, CKPT_DIR)            except Exception as e:                errors.append((f"Checkpoint {item['name']}", str(e)))        download_extras(ckpt_extra.value, CKPT_DIR)        # VAE        for cb, item in zip(vae_checks, vae_items):            if not cb.value: continue            try:                fetch(item, VAE_DIR)            except Exception as e:                errors.append((f"VAE {item['name']}", str(e)))        download_extras(vae_extra.value, VAE_DIR)        # ControlNet (SDXL)        for cb, item in zip(cn_checks, cn_items):            if not cb.value: continue            try:                fetch(item, CN_DIR)            except Exception as e:                errors.append((f"ControlNet {item['name']}", str(e)))        download_extras(cn_extra.value, CN_DIR)        # ADetailer        for cb, item in zip(ad_checks, ad_items):            if not cb.value: continue            try:                fetch(item, AD_DIR)            except Exception as e:                errors.append((f"ADetailer {item['name']}", str(e)))        download_extras(ad_extra.value, AD_DIR)        # ESRGAN        for cb, item in zip(esr_checks, esr_items):            if not cb.value: continue            try:                fetch(item, ESRGAN_DIR)            except Exception as e:                errors.append((f"ESRGAN {item['name']}", str(e)))        download_extras(esr_extra.value, ESRGAN_DIR)        # IP-Adapter (into ControlNet dir)        for cb, item in zip(ipad_checks, ipad_items):            if not cb.value: continue            try:                fetch(item, IPAD_DIR)            except Exception as e:                errors.append((f"IP-Adapter {item['name']}", str(e)))        download_extras(ipad_extra.value, IPAD_DIR)        # AnimateDiff (SDXL)        for cb, item in zip(anim_checks, anim_items):            if not cb.value: continue            try:                fetch(item, ANIM_DIR)            except Exception as e:                errors.append((f"AnimateDiff {item['name']}", str(e)))        download_extras(anim_extra.value, ANIM_DIR)        # ReActor (InsightFace)        for cb, item in zip(react_checks, react_items):            if not cb.value: continue            try:                fetch(item, REACT_DIR)            except Exception as e:                errors.append((f"ReActor {item['name']}", str(e)))        download_extras(react_extra.value, REACT_DIR)        # Embeddings        for cb, item in zip(emb_checks, emb_items):            if not cb.value: continue            try:                fetch(item, EMB_DIR)            except Exception as e:                errors.append((f"Embedding {item['name']}", str(e)))        download_extras(emb_extra.value, EMB_DIR)        # LoRA        for cb, item in zip(lora_checks, lora_items):            if not cb.value: continue            try:                fetch(item, LORA_DIR)            except Exception as e:                errors.append((f"LoRA {item['name']}", str(e)))        download_extras(lora_extra.value, LORA_DIR)        print("\n=== Summary ===")        if errors:            for name, msg in errors:                print("✖", name, "→", msg)        else:            print("✔ All downloads completed (check folders above).")btn.on_click(on_click)# --- Render UI ---display(Markdown("### SDXL Checkpoints"))for w in ckpt_checks: display(w)display(Markdown("**Extra URLs**")); display(ckpt_extra)display(Markdown("### SDXL VAE (multiple)"))for w in vae_checks: display(w)display(Markdown("**Extra URLs**")); display(vae_extra)display(Markdown("### ControlNet (SDXL)"))for w in cn_checks: display(w)display(Markdown("**Extra URLs**")); display(cn_extra)display(Markdown("### ADetailer models"))for w in ad_checks: display(w)display(Markdown("**Extra URLs**")); display(ad_extra)display(Markdown("### ESRGAN upscalers"))for w in esr_checks: display(w)display(Markdown("**Extra URLs**")); display(esr_extra)display(Markdown("### IP-Adapter (SDXL, stored in ControlNet folder)"))for w in ipad_checks: display(w)display(Markdown("**Extra URLs**")); display(ipad_extra)display(Markdown("### AnimateDiff (SDXL)"))for w in anim_checks: display(w)display(Markdown("**Extra URLs**")); display(anim_extra)display(Markdown("### ReActor (InsightFace)"))if not react_checks:    display(Markdown("_No defaults preselected. Paste your InsightFace/antelopev2 links below._"))display(Markdown("**Extra URLs**")); display(react_extra)display(Markdown("### Embeddings"))for w in emb_checks: display(w)display(Markdown("**Extra URLs**")); display(emb_extra)display(Markdown("### LoRA (SDXL)"))for w in lora_checks: display(w)display(Markdown("**Extra URLs**")); display(lora_extra)display(btn, out)

## Restart WebUI (optional)
Use this after new files are added so A1111 refreshes lists.

In [ ]:
import subprocess, time, shlextry:    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)    if out.returncode == 0 and "a1111:" in (out.stdout or ""):        print(out.stdout.strip())    else:        print("Supervisorctl unavailable, killing process…")        subprocess.run(["pkill", "-f", "launch.py"], check=False)        time.sleep(5)        print("✔ WebUI terminated, Supervisor will auto-restart it")    print("Wait 10–15 seconds, then refresh WebUI (7860).")except Exception as e:    print("⚠️ Restart failed:", e)    print("Manual command:")    print("supervisorctl -c /etc/supervisord.conf restart a1111")